In [2]:
pip install scikit-learn pandas numpy xgboost category_encoders

  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached category_encoders-2.6.3-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached statsmodels-0.14.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.5 kB)
Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cache

In [8]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy             as np
import pandas            as pd

from scipy.stats import zscore
#models
from xgboost                 import XGBClassifier

#metrics and assistants
from sklearn.metrics         import confusion_matrix, balanced_accuracy_score, precision_score, classification_report, recall_score, f1_score, make_scorer, cohen_kappa_score, roc_curve, auc, brier_score_loss, log_loss
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from category_encoders       import OneHotEncoder

In [23]:
df = pd.read_csv("../data/raw_data.csv")

In [24]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


In [15]:
enc = OneHotEncoder(cols=['person_home_ownership','loan_intent','cb_person_default_on_file'], use_cat_names=True)
df = enc.fit_transform(df)

In [18]:
df.head()

,person_age,person_income,person_home_ownership_RENT,person_home_ownership_OWN,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_emp_length,loan_intent_PERSONAL,loan_intent_EDUCATION,loan_intent_MEDICAL,...,loan_intent_HOMEIMPROVEMENT,loan_intent_DEBTCONSOLIDATION,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file_Y,cb_person_default_on_file_N,cb_person_cred_hist_length
0,22,59000,1,0,0,0,123.0,1,0,0,...,0,0,D,35000,16.02,1,0.59,1,0,3
1,21,9600,0,1,0,0,5.0,0,1,0,...,0,0,B,1000,11.14,0,0.10,0,1,2
2,25,9600,0,0,1,0,1.0,0,0,1,...,0,0,C,5500,12.87,1,0.57,0,1,3
3,23,65500,1,0,0,0,4.0,0,0,1,...,0,0,C,35000,15.23,1,0.53,0,1,2
4,24,54400,1,0,0,0,8.0,0,0,1,...,0,0,C,35000,14.27,1,0.55,1,0,4


In [19]:
df['loan_grade'] = df['loan_grade'].apply(lambda x: 1 if x == 'A' 
                                                    else 2 if x == 'B' 
                                                    else 3 if x == 'C' 
                                                    else 4 if x == 'D' 
                                                    else 5 if x == 'E' 
                                                    else 6 if x == 'F' 
                                                    else 7)

grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
self.prepared_data['loan_grade'] = self.prepared_data['loan_grade'].map(grade_mapping)

In [20]:
columns = df.columns.values
def substitute_character_in_list(strings_list, old_char, new_char):
    return [string.replace(old_char, new_char) for string in strings_list]

df.columns = substitute_character_in_list(columns,"_","")

In [21]:
df.head()

,personage,personincome,personhomeownershipRENT,personhomeownershipOWN,personhomeownershipMORTGAGE,personhomeownershipOTHER,personemplength,loanintentPERSONAL,loanintentEDUCATION,loanintentMEDICAL,...,loanintentHOMEIMPROVEMENT,loanintentDEBTCONSOLIDATION,loangrade,loanamnt,loanintrate,loanstatus,loanpercentincome,cbpersondefaultonfileY,cbpersondefaultonfileN,cbpersoncredhistlength
0,22,59000,1,0,0,0,123.0,1,0,0,...,0,0,4,35000,16.02,1,0.59,1,0,3
1,21,9600,0,1,0,0,5.0,0,1,0,...,0,0,2,1000,11.14,0,0.10,0,1,2
2,25,9600,0,0,1,0,1.0,0,0,1,...,0,0,3,5500,12.87,1,0.57,0,1,3
3,23,65500,1,0,0,0,4.0,0,0,1,...,0,0,3,35000,15.23,1,0.53,0,1,2
4,24,54400,1,0,0,0,8.0,0,0,1,...,0,0,3,35000,14.27,1,0.55,1,0,4


In [26]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('../')

In [3]:
import pytest

In [4]:
from src.data_prep import DataPrep

In [5]:
data = {
        'person_age':[25, 68, 35, 41, 30],
        'person_income':[33000, 88000, 25000, 56000, 75500],
        'person_home_ownership': ['OWN', 'RENT', 'OTHER', 'OWN', 'MORTGAGE'],
        'person_emp_length': [6.00, 15.00, 120.00, 20.00, 18.00],
        'loan_intent': ['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'VENTURE'],
        'loan_grade': ['A', 'B', 'C', 'D', 'E'],
        'loan_amount': [6000, 1500, 9000, 3200, 18000],
        'loan_int_rate': [16.2, 10.2, 15.8, 14.3, 15.4],
        'loan_status': [0, 1, 0, 1, 0],
        'loan_percent_income': [0.2, 0.05, 0.42, 0.08, 0.2],
        'cb_person_default_on_file': ['N', 'N', 'Y', 'N', 'Y'],
        'cb_person_cred_hist_length':[3, 6, 5, 4, 6]
    }
df_test = pd.DataFrame(data)

In [6]:
prep = DataPrep(None)

In [7]:
prep.raw_data=df_test

In [8]:
prep.encoder()
prep.loan_grade_prep()
prep.default_onfile_prep()
prep.subs_char_names()
df_final = prep.get_prepdata()

In [9]:
df_final

,personage,personincome,personhomeownershipOWN,personhomeownershipRENT,personhomeownershipOTHER,personhomeownershipMORTGAGE,personemplength,loanintentPERSONAL,loanintentEDUCATION,loanintentMEDICAL,loanintentVENTURE,loangrade,loanamount,loanintrate,loanstatus,loanpercentincome,cbpersondefaultonfile,cbpersoncredhistlength
0,25,33000,1,0,0,0,6.0,1,0,0,0,1,6000,16.2,0,0.20,0,3
1,68,88000,0,1,0,0,15.0,0,1,0,0,2,1500,10.2,1,0.05,0,6
2,35,25000,0,0,1,0,120.0,0,0,1,0,3,9000,15.8,0,0.42,1,5
3,41,56000,1,0,0,0,20.0,0,0,0,1,4,3200,14.3,1,0.08,0,4
4,30,75500,0,0,0,1,18.0,0,0,0,1,5,18000,15.4,0,0.20,1,6


In [12]:
df_dict = df_final.to_dict(orient='list')

df_dict

{'personage': [25, 68, 35, 41, 30],
 'personincome': [33000, 88000, 25000, 56000, 75500],
 'personhomeownershipOWN': [1, 0, 0, 1, 0],
 'personhomeownershipRENT': [0, 1, 0, 0, 0],
 'personhomeownershipOTHER': [0, 0, 1, 0, 0],
 'personhomeownershipMORTGAGE': [0, 0, 0, 0, 1],
 'personemplength': [6.0, 15.0, 120.0, 20.0, 18.0],
 'loanintentPERSONAL': [1, 0, 0, 0, 0],
 'loanintentEDUCATION': [0, 1, 0, 0, 0],
 'loanintentMEDICAL': [0, 0, 1, 0, 0],
 'loanintentVENTURE': [0, 0, 0, 1, 1],
 'loangrade': [1, 2, 3, 4, 5],
 'loanamount': [6000, 1500, 9000, 3200, 18000],
 'loanintrate': [16.2, 10.2, 15.8, 14.3, 15.4],
 'loanstatus': [0, 1, 0, 1, 0],
 'loanpercentincome': [0.2, 0.05, 0.42, 0.08, 0.2],
 'cbpersondefaultonfile': [0, 0, 1, 0, 1],
 'cbpersoncredhistlength': [3, 6, 5, 4, 6]}